In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
from transformers import BertTokenizer, BertModel
import json
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
torch.manual_seed(1)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cpu"

In [3]:
# #check if cuda is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

#### Data Loading

In [5]:
train_data = json.load(open('../Dataset/BIO_Tagged/ATE_train.json', 'r'))
test_data = json.load(open('../Dataset/BIO_Tagged/ATE_test.json', 'r'))
val_data = json.load(open('../Dataset/BIO_Tagged/ATE_val.json', 'r'))

In [6]:
word_to_idx = pickle.load(open('../Utils/word_to_idx.pkl', 'rb'))

In [7]:
tag_to_ix = pickle.load(open('../Utils/tag_to_ix.pkl', 'rb'))

#### RNN model

In [8]:
class RNN_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(RNN_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.rnn = nn.RNN(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        rnn_out, _ = self.rnn(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(rnn_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores


#### LSTM Model

In [9]:
class LSTM_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(LSTM_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### GRU model

In [10]:
class GRU_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(GRU_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.gru = nn.GRU(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        gru_out, _ = self.gru(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### Embedding mats

In [11]:
bert_embedding_mat = pickle.load(open('../Extracted Word Embeddings/bert_embedding_mat.pkl', 'rb'))
word2vec_embedding_mat = pickle.load(open('../Extracted Word Embeddings/word2vec_embedding_mat.pkl', 'rb'))
glove_embedding_mat = pickle.load(open('../Extracted Word Embeddings/glove_embedding_mat.pkl', 'rb'))

#### Glove + RNN 

In [12]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []


for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 954.03it/s] 


Train Loss: 0.5906751319996305, Val Loss: 0.48600818461751283, Train F1: 0.24792421500113568, Val F1: 0.4936049709652359
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 1280.75it/s]


Train Loss: 0.4745467263243056, Val Loss: 0.4407622191255495, Train F1: 0.5159481994997834, Val F1: 0.5871878967190712
Epoch:  2


...

Train Loss: 0.36156520150420945, Val Loss: 0.40679565660621475, Train F1: 0.6840069156304057, Val F1: 0.6536894304246933
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 864.15it/s]


Train Loss: 0.3525366788693019, Val Loss: 0.406747530771581, Train F1: 0.6943506197311087, Val F1: 0.6561248612243575
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 683.80it/s]

Train Loss: 0.3441290736494475, Val Loss: 0.40703572426416557, Train F1: 0.7053258764128988, Val F1: 0.6564892657316198


In [13]:
torch.save(rnn_model, 'Non Trainable Embeddings/Glove+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task 2/Saved Models/t2_RNN_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+RNN/val_f1.pkl', 'wb'))

#### Word2vec + RNN

In [14]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  1%|▏         | 12/906 [00:00<00:07, 119.09it/s]

100%|██████████| 219/219 [00:00<00:00, 682.58it/s]


Train Loss: 0.6518684866412586, Val Loss: 0.5325831061901023, Train F1: 0.19861567460388913, Val F1: 0.41638778399959125
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 652.28it/s]


...

Train Loss: 0.34138174529781534, Val Loss: 0.36205828336139795, Train F1: 0.6879242306632719, Val F1: 0.6802942154697792
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 1027.20it/s]


Train Loss: 0.3323538691035696, Val Loss: 0.3613403914283672, Train F1: 0.7002416810095701, Val F1: 0.6863871510307821
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 1000.80it/s]

Train Loss: 0.3242399464700593, Val Loss: 0.3615543705381487, Train F1: 0.7078857935966575, Val F1: 0.6902368501743693


In [15]:
torch.save(rnn_model, 'Non Trainable Embeddings/Word2vec+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task 2/Saved Models/t2_RNN_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+RNN/val_f1.pkl', 'wb'))

#### Bert + RNN

In [16]:
rnn_model = RNN_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 487.34it/s]


Train Loss: 0.5115846748531654, Val Loss: 0.4185662794875228, Train F1: 0.5117648202263461, Val F1: 0.6008823407211678
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 515.95it/s]


...

Train Loss: 0.3047435293775915, Val Loss: 0.3452359177172184, Train F1: 0.7284251571421084, Val F1: 0.6933911384251172
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 691.34it/s]


Train Loss: 0.292218828347674, Val Loss: 0.3461604512479479, Train F1: 0.7390190525020355, Val F1: 0.6932770945008352
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 459.05it/s]


Train Loss: 0.27994560464501184, Val Loss: 0.34846479016181814, Train F1: 0.7498621914033691, Val F1: 0.690814526254795


In [17]:
torch.save(rnn_model, 'Non Trainable Embeddings/Bert+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task 2/Saved Models/t2_RNN_Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+RNN/val_f1.pkl', 'wb'))

#### Glove + GRU

In [18]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 490.23it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 558.69it/s]


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 309.11it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 259.51it/s]

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [19]:
torch.save(gru_model, 'Non Trainable Embeddings/Glove+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task 2/Saved Models/t2_GRU_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+GRU/val_f1.pkl', 'wb'))

#### Word2vec + GRU

In [20]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  1%|          | 6/906 [00:00<00:17, 52.46it/s]

100%|██████████| 219/219 [00:00<00:00, 791.19it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 395.91it/s]


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 610.90it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 456.71it/s]

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [21]:
torch.save(gru_model, 'Non Trainable Embeddings/Word2vec+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task 2/Saved Models/t2_GRU_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+GRU/val_f1.pkl', 'wb'))

#### Bert + GRU

In [22]:
gru_model = GRU_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 445.61it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 561.95it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  2


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 594.98it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 296.18it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [23]:
torch.save(gru_model, 'Non Trainable Embeddings/Bert+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task 2/Saved Models/t2_GRU_Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+GRU/val_f1.pkl', 'wb'))

#### Glove + LSTM

In [24]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  2%|▏         | 14/906 [00:00<00:15, 58.81it/s]

100%|██████████| 219/219 [00:00<00:00, 621.45it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 547.95it/s]


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 548.56it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 659.07it/s]

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [25]:
torch.save(lstm_model, 'Non Trainable Embeddings/Glove+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task 2/Saved Models/t2_LSTM_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+LSTM/val_f1.pkl', 'wb'))

#### Word2vec + LSTM

In [26]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 603.92it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 675.37it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  2


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 606.86it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 300.78it/s]

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [27]:
torch.save(lstm_model, 'Non Trainable Embeddings/Word2vec+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task 2/Saved Models/t2_LSTM_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+LSTM/val_f1.pkl', 'wb'))

#### Bert + LSTM

In [28]:
lstm_model = LSTM_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 626.40it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 549.40it/s]


...

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 420.00it/s]


Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 309.05it/s]

Train Loss: 0.2489375187614501, Val Loss: 0.34846479016181814, Train F1: 0.7814219835995798, Val F1: 0.690814526254795


In [29]:
torch.save(lstm_model, 'Non Trainable Embeddings/Bert+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task 2/Saved Models/t2_LSTM_Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+LSTM/val_f1.pkl', 'wb'))